In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [5]:
df = pd.read_csv("Nat_Gas.csv")
df['Dates'] = pd.to_datetime(df['Dates'])
df = df.sort_values(by="Dates").reset_index(drop=True)



/tmp/ipython-input-2555469867.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [6]:
start_date = df['Dates'].min()
df['Months'] = df['Dates'].apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)


In [7]:
X = df['Months'].values.reshape(-1, 1)
y = df['Prices'].values
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [8]:
def estimate_price(date_str):
    date = pd.to_datetime(date_str, format="%Y-%m-%d")
    months_since_start = (date.year - start_date.year) * 12 + date.month - start_date.month
    return float(model.predict(np.array([[months_since_start]]))[0])

In [12]:
def _build_price_map(dates, prices):
    if prices is None:
        return {pd.to_datetime(d).normalize(): None for d in dates}
    if isinstance(prices, dict):
        return {pd.to_datetime(k).normalize(): float(v) for k, v in prices.items()}
    prices = list(prices)
    if len(prices) != len(dates):
        raise ValueError("Length of prices must match length of dates when prices is a list.")
    return {pd.to_datetime(d).normalize(): float(p) for d, p in zip(dates, prices)}


In [13]:
def price_storage_contract(
    injection_dates,
    withdrawal_dates,
    injection_prices=None,
    withdrawal_prices=None,
    injection_rate=float("inf"),
    withdrawal_rate=float("inf"),
    max_volume=float("inf"),
    storage_cost_per_unit_per_day=0.0,
    storage_cost_flat=0.0,
    process_same_day="inject_first"
):
    injections = [pd.to_datetime(d).normalize() for d in injection_dates]
    withdrawals = [pd.to_datetime(d).normalize() for d in withdrawal_dates]
    inj_price_map = _build_price_map(injections, injection_prices)
    wdr_price_map = _build_price_map(withdrawals, withdrawal_prices)

    events = []
    for d in injections:
        events.append({"date": d, "type": "inject", "price": inj_price_map.get(d)})
    for d in withdrawals:
        events.append({"date": d, "type": "withdraw", "price": wdr_price_map.get(d)})

    events.sort(key=lambda x: (x["date"], 0 if x["type"] == "inject" and process_same_day == "inject_first" else 1))

    volume = 0.0
    total_cost = 0.0
    total_revenue = 0.0
    storage_time_cost = 0.0
    details = []

    for i, ev in enumerate(events):
        date = ev["date"]
        price = ev["price"] if ev["price"] is not None else estimate_price(date)

        if ev["type"] == "inject":
            available_space = max_volume - volume
            to_inject = min(injection_rate, available_space)
            cost = to_inject * price
            volume += to_inject
            total_cost += cost
            details.append({"date": date, "action": "inject", "volume": to_inject, "price": price, "cash": -cost, "volume_after": volume})

        else:
            to_withdraw = min(withdrawal_rate, volume)
            revenue = to_withdraw * price
            volume -= to_withdraw
            total_revenue += revenue
            details.append({"date": date, "action": "withdraw", "volume": to_withdraw, "price": price, "cash": revenue, "volume_after": volume})

        if i < len(events) - 1:
            next_date = events[i+1]["date"]
            days = (next_date - date).days
            if days > 0 and storage_cost_per_unit_per_day:
                storage_time_cost += volume * storage_cost_per_unit_per_day * days

    storage_total = storage_time_cost + storage_cost_flat
    net_value = total_revenue - total_cost - storage_total
    status = "Profitable ✅" if net_value > 0 else "Unprofitable ❌"

    return {
        "Total Buy Cost": round(total_cost, 2),
        "Total Sell Revenue": round(total_revenue, 2),
        "Storage Costs (time-based)": round(storage_time_cost, 2),
        "Storage Costs (flat)": round(storage_cost_flat, 2),
        "Storage Costs (total)": round(storage_total, 2),
        "Net Contract Value": round(net_value, 2),
        "Status": status,
        "Details": pd.DataFrame(details)
    }


In [14]:
#Example
inj_dates = ["2023-06-30", "2023-07-31", "2023-08-31"]
wdr_dates = ["2023-09-30", "2023-10-31", "2023-11-30"]

result = price_storage_contract(
    inj_dates,
    wdr_dates,
    injection_rate=50,
    withdrawal_rate=50,
    max_volume=150,
    storage_cost_per_unit_per_day=0.0,
    storage_cost_flat=75.0
)

print(result["Status"], result["Net Contract Value"])
print(result["Details"])


Unprofitable ❌ -57.44
        date    action  volume      price        cash  volume_after
0 2023-06-30    inject      50  11.538696 -576.934795          50.0
1 2023-07-31    inject      50  11.577709 -578.885457         100.0
2 2023-08-31    inject      50  11.616722 -580.836120         150.0
3 2023-09-30  withdraw      50  11.655736  582.786782         100.0
4 2023-10-31  withdraw      50  11.694749  584.737444          50.0
5 2023-11-30  withdraw      50  11.733762  586.688106           0.0
